# Análisis de estado de ánimo de una persona basado en escuchas de Last.FM

En este trabajo se busca determinar el estado de ánimo de una persona, en la última semana, basándonos en sus escuchas registradas en la plataforma [Last.fm](https://www.last.fm/).

Para ello, seguiremos este proceso:
1. Obtener las canciones que ha escuchado el usuario en la última semana.
2. Obtener las letras de las canciones de la plataforma Genius.
3. Analizar los estados de ánimo predominantes en las letras de las canciones.
4. Visualizar los resultados con gráficas.

In [16]:
import argparse
import pathlib
import requests

from lastfm import lastfm_api
from genius import genius_api
from analysis import mood_analysis
from visualization import emotion_visualization

# To format the dictionaries.
from pprint import pprint

---------
## 1. Last.FM: Obtener las canciones que ha escuchado el usuario en la última semana
![Last.FM Logo](https://www.last.fm/static/images/logo_static.adb61955725c.png)

Para obtener las escuchas, primero necesitamos proveer un usuario de Last.FM válido.

In [8]:
user = input()
print(user)

Starkie785


In [9]:
# Check if the provided user exists.
print(f'Retrieving information from the user \'{user}\'')

userInfo = lastfm_api.user_info(user)

if 'error' in userInfo:
    print(f"{userInfo['message']} - {user}")
    exit()
else:
    print(f"The user '{user}' is a valid user.")

Retrieving information from the user 'Starkie785'
The user 'Starkie785' is a valid user.


A continuación, obtenemos las escuchas en la última semana, agrupadas por artista:

In [10]:
user_tracks = lastfm_api.user_tracks_from_lastweek(user)
songs_by_artist = lastfm_api.group_tracks_by_artist(user_tracks)

print(f"Found {len(user_tracks)} songs by {len(songs_by_artist)} different artists.")

Last.FM - Reading songs listened since '2021-03-14 00:00:00' by 'Starkie785'.
Last.FM - Read 50 out of 151 tracks.
Last.FM - Read 100 out of 151 tracks.
Last.FM - Read 150 out of 151 tracks.
Last.FM - Read 151 out of 151 tracks.
Found 151 songs by 26 different artists.


In [17]:
first_artist = list(songs_by_artist.keys())[0]
pprint(songs_by_artist[first_artist])

{'name': 'Architects',
 'songs': ['An Ordinary Extinction',
           'Black Lungs',
           'Meteor',
           'Impermanence (feat. Winston McCall)',
           'Giving Blood',
           'Goliath (feat. Simon Neil)',
           'Animals',
           'Little Wonder (feat. Mike Kerr)',
           'Do You Dream Of Armageddon?',
           'Demi God',
           'An Ordinary Extinction',
           'Modern Misery',
           'Impermanence (feat. Winston McCall)',
           'Holy Hell',
           'The Seventh Circle',
           'Giving Blood',
           'Black Lungs',
           'Do You Dream Of Armageddon?']}


-----
## 2. Genius - Obtener las letras de las canciones

![Genius Logo](https://www.edmsauce.com/wp-content/uploads/2019/06/genius-218x150.png)

Una vez hemos obtenido las escuchas, el siguiente paso es obtener las letras de las canciones. Usaremos la API de  [Genius](https://genius.com/), una base de datos de letras.

Las canciones instrumentales o aquellas cuya letra no encontremos serán ignoradas.

In [18]:
lyrics_by_artist = genius_api.lyrics_by_artists(songs_by_artist)

Genius - Searching lyrics of 'Architects'.
Genius - Searching lyrics of 'System of a Down'.
Genius - Searching lyrics of 'The Rolling Stones'.
Genius - Searching lyrics of 'Kiss'.
Genius - Searching lyrics of 'Robot Science'.
Genius - The song 'Robot Science - Fog' was not found.
Genius - Searching lyrics of 'Blind Guardian'.
Genius - Searching lyrics of 'Black Sabbath'.
Genius - Searching lyrics of 'Van Halen'.
Genius - Searching lyrics of 'Converge'.
Genius - Searching lyrics of 'TV on the Radio'.
Genius - Searching lyrics of 'Rammstein'.
Genius - Searching lyrics of 'Bon Jovi'.
Genius - Searching lyrics of 'Perturbator'.
Genius - The song 'Perturbator - Sexualizer' was not found.
Genius - Searching lyrics of 'Pendulum'.
Genius - The song 'Pendulum - Blood Sugar' was not found.
Genius - Searching lyrics of 'Spoon'.
Genius - Searching lyrics of 'Slayer'.
Genius - Searching lyrics of 'My Morning Jacket'.
Genius - Searching lyrics of 'Rise Against'.
Genius - Searching lyrics of 'Enter S

In [35]:
artist = lyrics_by_artist[0]
print(f"Name: {artist['name']}")

first_song = list(artist['songs'].keys())[0] 
print(f"Song: {first_song}\n")
print(artist['songs'][first_song])

Name: Architects
Song: Do You Dream Of Armageddon?

[Verse 1]
No tree can grow to Heaven
Till its roots reach down to Hell
Will we ever learn our lesson?
We all fall in parallel

[Verse 2]
No tree can grow to Heaven
Till its roots reach down to Hell
Will we ever learn our lesson?
We all fall in parallel
Don't bury the confession
There's a light behind the shell
Do you dream of Armageddon
Just to get yourself to Heaven?

[Outro]
We all fall in parallel
We all fall in parallel
We all fall in parallel
We all fall in parallеl
We all fall in parallel


--------------
## 3. NRCLex - Analisis de emociones de las letras

Para analizar las emociones de las letras de las canciones, vamos a utilizar la librería [NCRLex](https://github.com/metalcorebear/NRCLex). Esta librería toma de base la colección de palabras en inglés [EmoLex](https://saifmohammad.com/WebPages/NRC-Emotion-Lexicon.htm), compilada por la National Research Council (NRC) de Canada.

Esta colección asigna a las palabras que la componen una o más emociones del [modelo de Plutchik](https://en.wikipedia.org/wiki/Emotion_classification#Plutchik's_model). En concreto, las 8 emociones básicas: alegría, confianza, miedo, sorpresa, tristeza, aversión, ira y anticipación. Por ejemplo: `'abandoned': ['anger', 'fear', 'sadness']`


![Rueda de las emociones de Plutchik](https://upload.wikimedia.org/wikipedia/commons/thumb/c/ce/Plutchik-wheel.svg/600px-Plutchik-wheel.svg.png "Rueda de las emociones de Plutchik")


De cada una de las canciones, nos quedaremos **solo con las 3 emociones principales** y las devolveremos como un vector. 

Por ejemplo: `[0.0, 0.09230769230769231, 0.0, 0.0, 0.18461538461538463, 0.0, 0.0, 0.1076923076923077]`

In [36]:
song_emotions_by_artist = mood_analysis.analyse_lyrics(lyrics_by_artist)

In [44]:
artist = song_emotions_by_artist[1]
print(f"Name: {artist['name']}")

first_song = list(artist['songs'].keys())[0] 
print(f"Song: {first_song}\n")


print(song_emotions_by_artist[0]['categories'])
print(artist['songs'][first_song])

Name: Architects
Song: Do You Dream Of Armageddon?

['joy', 'trust', 'fear', 'surprise', 'sadness', 'disgust', 'anger', 'anticipation']
[0.0, 0.09230769230769231, 0.0, 0.0, 0.18461538461538463, 0.0, 0.0, 0.1076923076923077]
